# Imports and setup

In [ ]:
import os, sys
import numpy as np
import CL_SBI_mini as CL_SBI_mini

%load_ext autoreload

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('default')
plt.close('all')

# Specify dataset config parameters

In [ ]:
box = 2000
factor_kmin_cut = 4
kmax = 0.6
path_save_root = os.path.join("../", "DATASETS")

In [ ]:
dict_bound_augs = {
"eagle"     : dict(M_c=12.97, eta=-0.02, beta=0.28, M1_z0_cen=9.430, theta_out=0.09, theta_inn=-0.79, M_inn=13.42),
"illustris" : dict(M_c=14.99, eta=-0.23, beta=0.52, M1_z0_cen=10.75, theta_out=0.27, theta_inn=-1.99, M_inn=9.28 )
}

# Generate training set

In [ ]:
path_save = os.path.join(path_save_root, 'TRAIN')
NN_samples_cosmo = 8192
seed = 0

In [ ]:
hydro_key = 'eagle'
model_name = "Model_"+hydro_key
NN_samples_augs = 1
theta, xx, augs, ext_augs = CL_SBI_mini.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo=NN_samples_cosmo, NN_samples_augs=NN_samples_augs,
    dict_bounds_augs=dict_bound_augs[hydro_key], seed=seed, path_save=path_save, model_name=model_name,
    kmax=kmax, box=box, factor_kmin_cut=factor_kmin_cut
)

In [ ]:
print('theta.shape (#sampled cosmologies, #cosmological parameters) -->', theta.shape)
print('xx.shape (#sampled cosmologies, #baryonic models, #points k-space for Pk) -->', xx.shape)
print('augs.shape (#sampled cosmologies, #sweeped baryonic parameters) -->', augs.shape)
print('ext_augs.shape (#sampled cosmologies, #baryonic models, #baryonic parameters) -->', ext_augs.shape)

In [ ]:
hydro_key = 'illustris'
model_name = "Model_"+hydro_key
NN_samples_augs = 1
theta, xx, augs, ext_augs = CL_SBI_mini.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo=NN_samples_cosmo, NN_samples_augs=NN_samples_augs,
    dict_bounds_augs=dict_bound_augs[hydro_key], seed=seed, path_save=path_save, model_name=model_name,
    kmax=kmax, box=box, factor_kmin_cut=factor_kmin_cut
)

In [ ]:
print('theta.shape (#sampled cosmologies, #cosmological parameters) -->', theta.shape)
print('xx.shape (#sampled cosmologies, #baryonic models, #points k-space for Pk) -->', xx.shape)
print('augs.shape (#sampled cosmologies, #sweeped baryonic parameters) -->', augs.shape)
print('ext_augs.shape (#sampled cosmologies, #baryonic models, #baryonic parameters) -->', ext_augs.shape)

# Generate test set

In [ ]:
path_save = os.path.join(path_save_root, 'TEST')
NN_samples_cosmo = 2048
seed = 2

In [ ]:
hydro_key = 'vary_all'
model_name = "Model_"+hydro_key
NN_samples_augs = 20
theta, xx, augs, ext_augs = CL_SBI_mini.preprocessing_tools.generate_baccoemu_dataset(
    NN_samples_cosmo=NN_samples_cosmo, NN_samples_augs=NN_samples_augs,
    dict_bounds_augs=None, seed=seed, path_save=path_save, model_name=model_name,
    kmax=kmax, box=box, factor_kmin_cut=factor_kmin_cut
)

In [ ]:
print('theta.shape (#sampled cosmologies, #cosmological parameters) -->', theta.shape)
print('xx.shape (#sampled cosmologies, #baryonic models, #points k-space for Pk) -->', xx.shape)
print('augs.shape (#sampled cosmologies, #sweeped baryonic parameters) -->', augs.shape)
print('ext_augs.shape (#sampled cosmologies, #baryonic models, #baryonic parameters) -->', ext_augs.shape)

# Load stored datasets and validate them

In [ ]:
normalize = False
dset_dict = dict(
    add_noise_Pk = "cosmic_var_gauss",
    NN_noise_realizations = 10,
    kmax = kmax,
    box = box,
    factor_kmin_cut = factor_kmin_cut
)

theta, xx = CL_SBI_mini.load_datasets.load_and_preprocess_dset(
    path_load_dset = os.path.join("../", "DATASETS", "TRAIN"),
    list_model_names = ["Model_eagle", "Model_illustris"],
    dset_dict = dset_dict,
    normalize = normalize,
    path_save_norm = None,
    path_load_norm = None
)

In [ ]:
print('theta.shape (#sampled cosmologies x #baryonic models within list_model_names  x #noise realizations, #cosmological parameters) -->', theta.shape)
print('xx.shape (#sampled cosmologies x #baryonic models within list_model_names  x #noise realizations, #points k-space for Pk) -->', xx.shape)

In [ ]:
kf = 2.0 * np.pi / dset_dict["box"]
kmin=np.log10(dset_dict["factor_kmin_cut"]*kf)
N_kk = int((dset_dict["kmax"] - kmin) / (8*kf))
kk = np.logspace(kmin, kmax, num=N_kk)

N_plot_cosmo = 10
indexes = np.random.choice(xx.shape[0], N_plot_cosmo, replace=False)

fig, ax = mpl.pyplot.subplots(1,1,figsize=(9,6))
ax.set_ylabel(r'$P(k) \left[ \left(h^{-1} \mathrm{Mpc}\right)^{3} \right]$')
ax.set_xlabel(r'$\mathrm{Wavenumber}\, k \left[ h\, \mathrm{Mpc}^{-1} \right]$')
ax.axvline(10**kmax, c='k', ls=':', lw=1.)

ax.plot(np.array(kk), xx[indexes].T, c='limegreen', lw=1.5, marker=None, ms=2, alpha=0.7)
ax.set_xscale('log')
ax.set_yscale('log')

plt.show()

In [ ]:
normalize = True
dset_dict = dict(
    add_noise_Pk = "cosmic_var_gauss",
    NN_noise_realizations = 1,
    kmax = kmax,
    box = box,
    factor_kmin_cut = factor_kmin_cut
)

theta, xx = CL_SBI_mini.load_datasets.load_and_preprocess_dset(
    path_load_dset = os.path.join("../", "DATASETS", "TEST"),
    list_model_names = ["Model_vary_all"],
    dset_dict = dset_dict,
    normalize = normalize,
    path_save_norm = os.path.join("../", "DATASETS", "TEST"),
    path_load_norm = None
)

In [ ]:
print('theta.shape (#sampled cosmologies x #baryonic models within list_model_names  x #noise realizations, #cosmological parameters) -->', theta.shape)
print('xx.shape (#sampled cosmologies x #baryonic models within list_model_names  x #noise realizations, #points k-space for Pk) -->', xx.shape)

In [ ]:
kf = 2.0 * np.pi / dset_dict["box"]
kmin=np.log10(dset_dict["factor_kmin_cut"]*kf)
N_kk = int((dset_dict["kmax"] - kmin) / (8*kf))

N_plot_cosmo = 10
indexes = np.random.choice(xx.shape[0], N_plot_cosmo, replace=False)

fig, ax = mpl.pyplot.subplots(1,1,figsize=(9,6))
ax.set_ylabel(r'$\mathrm{Norm}\left(P(k) \left[ \left(h^{-1} \mathrm{Mpc}\right)^{3} \right]\right)$')
ax.set_xlabel('$k - index [adim]$')

ax.plot(np.arange(N_kk), xx[indexes].T, c='grey', lw=1.5, marker=None, ms=2, alpha=0.7)

plt.show()